In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# https://github.com/matplotlib/ipympl/blob/main/docs/examples/full-example.ipynb
# https://kapernikov.com/ipywidgets-with-matplotlib/
# https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html
# https://www.statology.org/pandas-subplots/

In [ ]:
def read(file):
    _df = pd.read_csv(file, date_format='%d.%m.%Y, %H:%M:%S', index_col ="Timestamp")
    _df['energy'] = _df['Energy in Wm (MRAM)'] / 60 / 1000
    return _df['energy']

folder_path = './data'
files = filter(lambda f: not f.startswith('.'), os.listdir(folder_path))
frames = map(read, map(lambda f: folder_path + '/' + f, files))
df = pd.concat(frames)

In [ ]:
%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

date_format = '%Y-%m-%d'
tomorrow = datetime.now() + timedelta(days=1)
past = tomorrow - timedelta(weeks=1, days=1)

start_date = past.strftime(date_format)
end_date = tomorrow.strftime(date_format)
resample = '1D'

presets = [
    (0, 'Today', datetime.now(), datetime.now() + timedelta(days=1), '15T'),
    (1, 'Yesterday', datetime.now() - timedelta(days=1), datetime.now(), '15T'),
    (2, 'Last Week', datetime.now() - timedelta(days=7), datetime.now(), '1D'),
    (3, 'Last Week (incl. Today)', datetime.now() - timedelta(days=7), datetime.now() + timedelta(days=1), '1D'),
    (4, 'Last Month', datetime.now() - timedelta(days=30), datetime.now(), '1D'),
]
presets_picker = widgets.Dropdown(
    description='Presets:',
    value=3,
    options=map(lambda d: (d[1], d[0]), presets),
)
from_picker = widgets.DatePicker(
    description='From',
    value=datetime.strptime(start_date, date_format),
    disabled=False
)
to_picker = widgets.DatePicker(
    description='To',
    value=datetime.strptime(end_date, date_format),
    disabled=False
)
resample_picker = widgets.Dropdown(
    description='Resample',
    value=resample, 
    options=['5T', '10T', '15T', '30T', '1H', '6H', '12H', '1D', '1W'], 
)
output_detail = widgets.HTML()
output_plot = widgets.Output()


def filter_df(df, start_date, end_date, resample):
    _df = df.loc[(df.index >= pd.Timestamp(start_date)) & (df.index <= pd.Timestamp(end_date))]
    return _df.resample(resample).sum()

def get_details_table(data):
    rows = map(
        lambda d: f"""
        <tr>
            <th><b>{d[0]}</b></th>
            <td>{d[1]:.4f} kWh</td>
        </tr>
        """, 
        data
    )
    return f"<table>{''.join(rows)}</table>"

def print_df_details(df):
    details = [
        ('Sum', df.sum()),
        ('Median', df.median()),
        ('Mean', df.mean()),
        ('Standard Deviation', df.std()),
        ('Minimum', df.min()),
        ('Maximum', df.max()),
    ]
    output_detail.value = get_details_table(details)

def draw_df(df):
    ax.clear()
    ax.autoscale()
    ax.set_title('PV Energy')
    ax.set_xlabel('Timestamp')
    ax.set_ylabel('Energy [kWh]')
    ax.grid(True)
    ax.set_ylim(bottom=0, auto=True)
    ax.plot(df, color='C0', marker='o')

def update():
    _df = filter_df(df, start_date, end_date, resample)
    print_df_details(_df)
    draw_df(_df)


def update_presets(value):
    global start_date
    global end_date
    global resample
    
    selected = presets[value]

    from_picker.value = selected[2]
    start_date = selected[2].strftime(date_format)

    to_picker.value = selected[3]
    end_date = selected[3].strftime(date_format)

    resample_picker.value = selected[4]
    resample = selected[4]

    update()
presets_picker.observe(lambda v: update_presets(v.new), 'value')

def update_from(value):
    global start_date
    start_date = str(value)
    update()
from_picker.observe(lambda v: update_from(v.new), 'value')

def update_to(value):
    global end_date
    end_date = str(value)
    update()
to_picker.observe(lambda v: update_to(v.new), 'value')

def update_resample(value):
    global resample
    resample = str(value)
    update()
resample_picker.observe(lambda v: update_resample(v.new), 'value')


with output_plot:
    fig, ax = plt.subplots(constrained_layout=True, figsize=(16, 8))
    fig.canvas.toolbar_visible = False
    fig.canvas.header_visible = False
    fig.canvas.footer_visible = False
    fig.canvas.resizable = False

update_presets(3)

controls = widgets.VBox([presets_picker, widgets.Label(), from_picker, to_picker, resample_picker])
header = widgets.HBox([controls, output_detail])
header.layout = widgets.Layout(
        width='70%',
        display='flex',
        justify_content='space-around',
        align_items='center'
)
widgets.VBox([output_plot, header])